수업때 vs code말고 이걸로 함

# 한페이지의 댓글 가져오기
- 크롬브라우저 띄우기
- 식당 페이지로 이동
- 댓글 더보기 버튼 클릭
- 댓글 가져오기
- 브라우저 종료

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

In [2]:
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d3200324-Reviews-or50-Jungsik-Seoul.html'

#driver 생성 - 웹브라우저 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(5)
# 브라우저 크기 지정
print(driver.get_window_size())
# driver.set_window_size(width=1920, height=1080)
driver.maximize_window()
#이동
driver.get(url)
# 더보기 버튼을 찾아서 클릭
# <span class='taLnk  ulBlueLinks'> 클래스를 두개 설정. (공백이 구분자임.)
more_btn = driver.find_elements(By.CSS_SELECTOR, 'span.taLnk.ulBlueLinks')
print(more_btn[0].text)

# 더보기 링크가 활성화(클릭가능상태) 될때까지 기다린다. 
time.sleep(1)
more_btn[0].click() #클릭 동작.

time.sleep(0.5) #더보기 클릭후 모든 댓글을 받아올때 까지 기다린다.

# BeautifulSoup을 이용해 댓글 가져오기
# 전체 페이지 조회
# driver.page_source
soup = BeautifulSoup(driver.page_source, 'lxml')
# 댓글 1개  css: p.partial_entry
comment_tag_list = soup.select('p.partial_entry')
comment_list = [] #각 댓글을 저장.

for tag in comment_tag_list:
    comment_list.append(tag.get_text().strip())
    
print(comment_list)    
driver.close()

{'width': 1051, 'height': 806}
더 보기
['들어가는 순간부터 서비스는 나를 편안하게 해주었다. 요리에 대한 부가 설명이 끝까지 명확하고 간결하게 이루어 졌으며, 나의 분자요리에 대한 안좋은 인식을 깨 주었다.', '아기자기한 플레이팅이 돋보이고 돌하루방 아이스크림 등 창의력인 디저트도 선보이는 곳입니다. 한번쯤 가볼만한 곳으로, 화이트 무드의 레스토랑 분위기는 여자분들이 충분히 좋아할 법 합니다~', '부모님 결혼기념일에 다녀온 정식당항상 사람이 많은 곳이라 3주전 미리 예약을 하고 다녀왔습니다.뉴욕에선 미슐랭 2스타를 받은 쉐프가 하는 곳이라 더욱 신뢰가 갔습니다.가보니 맛도 좋았고 서비스도 최고였습니다.기념일이라고하니 사진과같이 소소한 이벤트도 해주고 좋은 추억이 되었습니다.', '교수들이 학위심사턱으로 주로 간다는 정식당. 여기도 여지 없이 김영란법 적용으로 조만간 문 닫을 수도 있을것 같다는. 꽤 고급스런 음식 프리젠테이션때문인지, 대접받는다는 느낌이 들었다. 퓨전 음식이라, 비교대상이 없어서 맛있다 말하긴 좀 거시기하다. 다만, 맛없진 않았다. 새로운 형식의, 모양의, 스타일의 한식이었다고나 할까. 이곳은 절대 맛으로 가는 곳은 아니다. 상대에게 대접했다는 느낌을 줘야만 하는 상황이 아니라면 굳이 내 돈 주고 날 위해 갈것 같지 않다.', '방대한 와인리스트 (다 있는지는 모르겠으나)직원들의 서비스 질도 나쁘지 않고..플레이팅과 맛도 정갈합니다만......두번째가니 모든 느낌이 절반으로 줄어드는 곳이군요.', '8월초 토요일에 런치메뉴를 먹었습니다. 주말은 예약이 늘 차있어서 약 3주전에 예약해야 갈 수 있었어요. 총 5가지 코스가 나왔고 평소 싱겁게 먹는편도 아닌데 간이 짜다는 느낌이 상당했네요. (4코스,5코스 중 선택할 수 있습니다.)서비스는 딱딱하고 건조했어요. 정말 안좋았던 기억은, 마지막 디저트였던 파이를 포크로 가르는 순간 엄청 긴 머리카락이 나왔습니다. 일행이 파이를 시켰고 저는 돌하르방 아이스크림을 선택했는데 

In [4]:
comment_list[0]

'들어가는 순간부터 서비스는 나를 편안하게 해주었다. 요리에 대한 부가 설명이 끝까지 명확하고 간결하게 이루어 졌으며, 나의 분자요리에 대한 안좋은 인식을 깨 주었다.'

In [13]:
len(comment_list)

10

# 댓글 페이지 처리 
- 전체 댓글 다 가져오기
- 마지막 페이지 번호를 가져온다.

## 코드순서
- 한페이지의 댓글을 읽은 뒤(1. 더보기 버튼 클릭, 2. 댓글 저장) 다음 버튼을 클릭해서 페이지 이동.
- 마지막 페이지에서는 다음 버튼이 disabled 되어 click() 시 Exception 발생
    - Exception 발생하면 크롤링을 멈춘다.

## 페이징

In [2]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d3200324-Reviews-or50-Jungsik-Seoul.html'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
# options.add_argument("disable-gpu") #이건 해결된 듯. 크롬 60에서 문제


service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)#, options=options)
driver.implicitly_wait(5)
driver.get(url)
# wait = WebDriverWait(driver, 10)
i = 0
comment_list = []
while True:
    i+=1
    print(i, end=' ')
    # 더보기 링크 클릭
    more_btn = WebDriverWait(driver, 10).until(EC.presence_of_element_located([By.CSS_SELECTOR, 'span.taLnk.ulBlueLinks']))
#     more_btn = driver.find_element(By.CSS_SELECTOR, ".taLnk.ulBlueLinks")
    more_btn.click()
    # 댓글 조회
    time.sleep(1) #댓글 내용이 모두 나오길 기다린다.
    # 이미 있던 거라서 WebDriverWait()가 의미없다. 이건 그냥 시간 줘서 기다려야 한다.
    comment_tag_list = driver.find_elements(By.CSS_SELECTOR, 'p.partial_entry')
    for tag in comment_tag_list:
        comment_list.append(tag.text)  #댓글 텍스트 조회
    
    # 다음 버튼 클릭 : a.nav.next.taLnk.ui_button.primary
    try:
        # 이것도 버튼은 이미 있는 상태이므로 WebDriverWait() 안 써도 된다.
        next_btn = driver.find_element(By.CSS_SELECTOR, 'a.nav.next.ui_button')
        # 마지막 버튼은 disabled  상태여서 클릭이 안되서 ElementClickInterceptedException 발생한다.
        next_btn.click()
        
    except:
        print('종료 while')
        break
    
    time.sleep(1) #다음 페이지로 이동할 때 까지의 기다린다.
    
# driver.close()    #처리를 확인후 다음 cell에서 close. 최종본에는 여기서.

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 종료 while


In [5]:
print(len(comment_list))
print(comment_list[0])
print()
print(comment_list[10])
print()
print(comment_list[100])

167
들어가는 순간부터 서비스는 나를 편안하게 해주었다. 요리에 대한 부가 설명이 끝까지 명확하고 간결하게 이루어 졌으며, 나의 분자요리에 대한 안좋은 인식을 깨 주었다.

공간이 좁은 감은 있습니다 인테리어가 크게 세련되지 않습니다 식단보가 가죽 비슷한 건데 좋게 생각되지 않았습니다 하지만 음식은 좋았습니다 .

작년, 베를린에 있습니다. 나는 내가 방문한 레스토랑 중 한 세계 최고의 레스토랑이 있습니다. 우리는 꽤 실망스러웠습니다. 서울에 있는. 저와 제 딸은 에서 점심을 먹기로 jungsik, 아시아 최고의 음식점 중 한 두번째입니다.

호텔로 들어가는 문, 입력한 고요한 환경, 정신없는 도시에서 멀리 떨어진 느낌 탤컴파우더. 직원들은 친절하지 않은 않습니다. 요리사를 볼 수 있고 수시로 체크하고 주방에 초점을 그의 노래 에 빠졌습니다.

선택한 코스는 한시도 저렴한 점심 메뉴는 다섯 가집니다. 에피타이저가 나오는 식사는 정말 소개. 놀라 울 정도로 창조적 이고 맛있는 생선회 의 시작이었다. 나는 바다 가재 를 선택하는 과정 (수 있는 추가 비용) 나 물고기에 대한 생각... 멋진 맛의. 물론 우리의 고기는 오리 인데 해본 최고의 요리 중 하나. 식사와 나쁜 아무것도 말할 수 있는 아주 좋았어요.


```python
# 더보기버튼 클릭 - 여러번 요청처리 코드
    attempt = 0
    while attempt != 3:  #최대시도횟수
        try:
            more_btn = wait.until(EC.presence_of_all_elements_located([By.CSS_SELECTOR, 'span.taLnk.ulBlueLinks']))
             more_btn[0].click()
             break  #클릭을 정상처리한 경우 더 시도 하지 않고 반복문 나온다.
         except:
             #클릭실패 - 잠시 기다렸다 다시 시도.
             time.sleep(0.5)
             print("더보기 클릭 시도 횟수:", attempt)
             attempt += 1
```

### 클릭 처리 함수로 만듬

In [1]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
def retryMoreBtnClick(driver, selector, max_try=3):
    """
    더보기 버튼 클릭 (이건 여러개이므로 find_elements_xx로 조회)
    예외 - 클릭 실패하면 Exception발생
    """
    attempt = 0
    wait = WebDriverWait(driver, 10)
    while attempt < max_try:
        try:
            tags = wait.until(EC.presence_of_all_elements_located([By.CSS_SELECTOR, selector]))
            tags[0].click()
            return #클릭성공하면 return
        except Exception as e:
            time.sleep(1)
            print('retryMoreBtnClick: 재시도횟수 - %d' % attempt)
            print(e)
            attempt+=1
            
    #여기까지 왔단건 실패한 것이므로 예외발생
    raise Exception("클릭하지 못했습니다.")

In [14]:
def retryNextBtnClick(driver, selector, max_try=3):
    """
    다음 버튼 클릭 (이건 여러개이므로 find_element_xx로 조회)
    예외 - 클릭 실패하면 Exception발생
    """
    attempt = 0
    while attempt < max_try:
        try:
            tag = wait.until(EC.presence_of_element_located([By.CSS_SELECTOR, selector]))
            tag.click()
            return
        except Exception as e:
            time.sleep(1)
            print('retryNextBtnClick: 재시도횟수 - %d' % attempt)
            print(e)
            attempt+=1
    
    raise Exception("클릭하지 못했습니다.")

In [15]:
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d3200324-Reviews-or50-Jungsik-Seoul.html'


driver = Chrome()

# options = ChromeOptions()
# options.add_argument('headless')
# #화면사이즈 변경
# options.add_argument('window-size=1920x1080')
# driver = Chrome(options=options)


driver.get(url)
wait = WebDriverWait(driver, 10)
#마지막 페이지 번호 - div.pageNumbers > a:last-child
a_tag = wait.until(EC.presence_of_element_located([By.CSS_SELECTOR,'div.pageNumbers > a:last-child']))
print(a_tag.text)

last_page = int(a_tag.text.strip())
#댓글들을 저장할 리스트
comment_list = []
for i in range(last_page-1):
    
    # 더보기 링크 클릭
    try:
        retryMoreBtnClick(driver, 'span.taLnk.ulBlueLinks', 5)

        # 댓글 조회
        time.sleep(1) #댓글 내용이 모두 나오길 기다린다.
        comment_tag_list = driver.find_elements_by_css_selector('p.partial_entry')
        for tag in comment_tag_list:
            comment_list.append(tag.text)  #댓글 텍스트 조회

        # 다음 버튼 클릭 : a.nav.next.taLnk.ui_button.primary
        retryNextBtnClick(driver,'a.nav.next.taLnk.ui_button.primary', 5)

        print(i, end=' ')
        time.sleep(1) #다음 페이지로 이동할 때 까지의 기다린다.
    except Exception as e:
        raise e
#         continue
        
#for문 종료
driver.close()    

22
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 retryMoreBtnClick: 재시도횟수 - 0
Message: element click intercepted: Element <span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickCollapse',event,this);">...</span> is not clickable at point (168, 8). Other element would receive the click: <a class="ui_tab active" data-tab-name="...">리뷰</a>
  (Session info: chrome=78.0.3904.97)

retryMoreBtnClick: 재시도횟수 - 1
Message: element click intercepted: Element <span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickCollapse',event,this);">...</span> is not clickable at point (168, 8). Other element would receive the click: <a class="ui_tab active" data-tab-name="...">리뷰</a>
  (Session info: chrome=78.0.3904.97)

retryMoreBtnClick: 재시도횟수 - 2
Message: element click intercepted: Element <span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickCollapse',event,this);">...</span> is not clickable at point (168, 8). Other element would receive the click: <a class="ui_tab a

Exception: 클릭하지 못했습니다.